# Bitácora de trabajo Rizosfera

Initialize Qiime2:

In [1]:
source activate qiime2-2019.10

(qiime2-2019.10) 

: 1

Visualization of Feature table stats:

In [4]:
qiime feature-table summarize \
  --i-table table-rh.qza \
  --o-visualization table-rh.qzv \
  --m-sample-metadata-file metadata-rh.tsv

Saved Visualization to: table-rh.qzv
(qiime2-2019.10) 

: 1

In [5]:
qiime tools view table-rh.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.(qiime2-2019.10) 

## 2020-03-05 Rhizosphere taxonomic classification

We created the folder `classifier-rhizosphere/` in the Bios server, inside that folder are the files `rep-seqs.qza` corresponding to the rhizosphere reads and the classifier `classifier.qza`.

Then we ran the classifier:

```sh
screen
time qiime feature-classifier classify-sklearn \
  --i-classifier classifier.qza \
  --i-reads rep-seqs.qza \
  --o-classification taxonomy-rhizosphere.qza
```

Export taxonomy to a TSV file:

In [8]:
qiime tools export \
    --input-path classifier/taxonomy-rh.qza \
    --output-path classifier/

Exported classifier/taxonomy-rh.qza as TSVTaxonomyDirectoryFormat to directory classifier/
(qiime2-2019.10) 

: 1

## Phylogenetic inference

We aligned the V3-V4 16S sequences by MAFFT in the BIOS server:

```bash
cd rhizosphere
mkdir phylogenetic-tree
cd phylogenetic-tree

qiime alignment mafft \
  --i-sequences ../rep-seqs-rh.qza \
  --o-alignment aligned-rep-seqs-rh.qza
```
  
Then we did the masking to remove non-informative positions:

```bash
qiime alignment mask \
  --i-alignment aligned-rep-seqs-rh.qza \
  --o-masked-alignment masked-aligned-rep-seqs-rh.qza
```
  
Then we generated the tree with FastTree:

```bash
qiime phylogeny fasttree \
  --i-alignment masked-aligned-rep-seqs-rh.qza \
  --o-tree fasttree-tree-rh.qza
```
  
Finally we rooted the tree:

```bash
qiime phylogeny midpoint-root \
  --i-tree fasttree-tree-rh.qza \
  --o-rooted-tree fasttree-tree-rooted-rh.qza
```
  
## Classification Barplot


```bash
cd Documentos/genomeseq/rhizosphere/classifier/

qiime taxa barplot \
  --i-table table-rh.qza \
  --i-taxonomy taxonomy-rhizosphere.qza \
  --m-metadata-file ../metadata_rhizosphere.tsv \
  --o-visualization taxa-bar-plots-rh.qzv
```

In [2]:
qiime tools view classifier/taxa-bar-plots-rh.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.(qiime2-2019.10) 

## 2020-03-18 Reunión por Meet, pasos a seguir

Crear gráficas que permitan ir escribiendo el artículo. La gráfica debe explicarse a sí misma.

### Figuras publicación

Se creó un archivo de LibreOffice Impress con las figuras tentativas y su respectiva leyenda.

## 2020-03-25

Estefany me mandó la tabla de ASVs en formato .tsv, así que en el servidor de BIOS la convertí a .biom mediante el siguiente comando:

```bash
(qiime2-2019.10) jeperezj@r1masterl01:~/jcar/rhizosphere$ biom convert -i rhizosphere_asv.tsv -o table-rh.biom --to-hdf5
```

Luego convertí el .biom en un .qza:
```bash
(qiime2-2019.10) jeperezj@r1masterl01:~/jcar/rhizosphere$ qiime tools import \
    --input-path table-rh.biom \
    --type 'FeatureTable[Frequency]' \
    --output-path feature-table-rh.qza
```
    

## Composition analysis (ANCOM)

We are also analysing composition in order to know which ASVs have different abundances in the different samples. We ran the following commands:

```bash
(qiime2-2019.10) jeperezj@r1masterl01:~/jcar/rhizosphere/ancom-composition-analysis$ qiime composition add-pseudocount \
    --i-table feature-table-rh.qza \
    --o-composition-table comp-table-rh.qza
    
(qiime2-2019.10) jeperezj@r1masterl01:~/jcar/rhizosphere/ancom-composition-analysis$ time qiime composition ancom \
    --i-table comp-table-rh.qza \
    --m-metadata-file ../metadata_rhizosphere.tsv \
    --m-metadata-column Plant_status \
    --o-visualization ancom-plant-status-rh.qzv
```

The last commands takes a lot of time to run (even in the BIOS server) and has not yet worked.

**2020-03-27:** The command worked after fixing the metadata file interchaging the fist two columns.

The result was as follows:

![ancom-volcano-plot-plant-status-rh-taxa.png](ancom-composition-analysis/ancom-volcano-plot-plant-status-rh-taxa.png)

In [2]:
qiime tools view ancom-composition-analysis/ancom-plant-status-rh.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.(qiime2-2019.10) 

## ANCOM with only cultured soil

As we expect a great change in taxonomic entities between cultured and uncultured soil, is better to create a new data set containing only data from cultured soil and compare Modern and Ancestral samples:

```bash
(qiime2-2019.10) jeperezj@r1masterl01:~/jcar/rhizosphere/ancom-composition-analysis$ qiime feature-table filter-samples \
    --i-table feature-table-rh.qza \
    --m-metadata-file metadata_rhizosphere.tsv \
    --p-where "[Plant_status]!='NA'" \
    --o-filtered-table only-cultured-soil-ancom/cultured-soil-table--rh.qza

```

Then we ran the ANCOM again:

```bash
(qiime2-2019.10) jeperezj@r1masterl01:~/jcar/rhizosphere/ancom-composition-analysis/only-cultured-soil-ancom$ qiime composition add-pseudocount \
    --i-table cultured-soil-table--rh.qza \
    --o-composition-table comp-table-cultured-rh.qza

(qiime2-2019.10) jeperezj@r1masterl01:~/jcar/rhizosphere/ancom-composition-analysis/only-cultured-soil-ancom$ time qiime composition ancom \
    --i-table comp-table-cultured-rh.qza \
    --m-metadata-file ../metadata_rhizosphere.tsv \
    --m-metadata-column Plant_status \
    --o-visualization ancom-plant-status-cultured-rh.qzv
```
This was the result:

![ancom-volcano-plot-plant-status-cultured-rh-taxa.png](ancom-composition-analysis/ancom-volcano-plot-plant-status-cultured-rh-taxa.png)

In [3]:
qiime tools view ancom-composition-analysis/ancom-plant-status-cultured-rh.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.(qiime2-2019.10) 

## Visualization of taxonomic classification:

```bash
(qiime2-2019.10) nesper@gojira-E402MA:~/Documentos/genomeseq/rhizosphere/classifier$ qiime metadata tabulate \
    --m-input-file taxonomy-rhizosphere.qza \
    --o-visualization taxonomy-rhizosphere.qzv
```

## Family level ANCOM

We grouped ASVs by family and ran the ANCOM analysis only for cultured soil at family level:

```bash
(qiime2-2019.10) jeperezj@r1masterl01:~/jcar/rhizosphere/ancom-composition-analysis/collapsed-to-family$ qiime taxa collapse \
    --i-table cultured-soil-table-rh.qza \
    --i-taxonomy taxonomy-rhizosphere.qza \
    --o-collapsed-table table-family-level-rh.qza \
    --p-level 5
    
(qiime2-2019.10) jeperezj@r1masterl01:~/jcar/rhizosphere/ancom-composition-analysis/collapsed-to-family$ qiime composition add-pseudocount \
    --i-table table-family-level-rh.qza \
    --o-composition-table comp-table-famlvl-rh.qza

(qiime2-2019.10) jeperezj@r1masterl01:~/jcar/rhizosphere/ancom-composition-analysis/collapsed-to-family$ time qiime composition ancom \
    --i-table comp-table-famlvl-rh.qza \
    --m-metadata-file ../metadata_rhizosphere.tsv \
    --m-metadata-column Plant_status \
    --o-visualization ancom-famlvl-cult-rh.qzv
```

Visualization of ANCOM at family level:

In [2]:
qiime tools view ancom-composition-analysis/ancom-famlvl-cult-rh.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.(qiime2-2019.10) 

Visualization:

![](ancom-composition-analysis/ancom-volcano-plot-plant-status-cultured-famlvl-rh.png)

The group with differential abundance was:

D_0__Bacteria;D_1__FBP;D_2__metagenome;D_3__metagenome;D_4__metagenome

What family could be that?

## 2020-04-17 Phylogenetic tree with abundance-filtered Feature table

In order to produce a more clean phylogenetic tree we are going to remove features with less than 10 observations.
We now proceed to filter the feature table and the rep-seqs accordingly:

In [2]:
mkdir filtered-data

qiime feature-table filter-features \
    --i-table table-rh.qza \
    --p-min-frequency 10 \
    --o-filtered-table filtered-data/filtered-table.qza

qiime feature-table filter-seqs \
  --i-data rep-seqs-rh.qza \
  --i-table filtered-data/filtered-table.qza \
  --o-filtered-data filtered-data/filtered-rep-seqs.qza

(qiime2-2019.10) (qiime2-2019.10) Saved FeatureTable[Frequency] to: filtered-data/filtered-table.qza
(qiime2-2019.10) (qiime2-2019.10) Saved FeatureData[Sequence] to: filtered-data/filtered-rep-seqs.qza
(qiime2-2019.10) 

: 1

Produce the phylogenetic tree with filtered data:

In [3]:
cd filtered-data

qiime alignment mafft \
  --i-sequences filtered-rep-seqs.qza \
  --o-alignment aligned-filtered-rep-seqs.qza

qiime alignment mask \
  --i-alignment aligned-filtered-rep-seqs.qza \
  --o-masked-alignment masked-aligned-filtered-rep-seqs.qza

qiime phylogeny fasttree \
  --i-alignment masked-aligned-filtered-rep-seqs.qza \
  --o-tree unrooted-tree-filtered.qza
  
cd ../

(qiime2-2019.10) (qiime2-2019.10) Saved FeatureData[AlignedSequence] to: aligned-filtered-rep-seqs.qza
(qiime2-2019.10) (qiime2-2019.10) Saved FeatureData[AlignedSequence] to: masked-aligned-filtered-rep-seqs.qza
(qiime2-2019.10) (qiime2-2019.10) Saved Phylogeny[Unrooted] to: unrooted-tree-filtered.qza
(qiime2-2019.10) (qiime2-2019.10) (qiime2-2019.10) 

: 1

Group samples by Plant_status:

In [5]:
qiime feature-table group \
    --i-table filtered-data/filtered-table.qza \
    --p-axis sample \
    --m-metadata-file metadata-rh.tsv \
    --m-metadata-column Plant_status \
    --p-mode sum \
    --o-grouped-table filtered-data/grouped-filtered-table.qza

Saved FeatureTable[Frequency] to: filtered-data/grouped-filtered-table.qza
(qiime2-2019.10) 

: 1

The resulting tree was uploaded to iToL and exported from there. This is the result:
![](itol-tree/tree.svg)

## 2020-05-13 UniFrac distance matrix

We measured beta-diversity using weighted UniFrac:

In [9]:
mkdir -p beta-diversity/unifrac

qiime diversity beta-phylogenetic \
    --i-table table.qza \
    --i-phylogeny fasttree-tree-rooted-rh.qza \
    --p-metric weighted_unifrac \
    --o-distance-matrix beta-diversity/unifrac/weighted-unifrac-rh.qza

(qiime2-2019.10) (qiime2-2019.10) Saved DistanceMatrix % Properties('phylogenetic') to: beta-diversity/unifrac/weighted-unifrac-rh.qza
(qiime2-2019.10) 

: 1

We also performed a PERMANOVA using the `beta-group-significance` command to see if samples within a group are more similar between them than they are to samples from the other groups:

In [10]:
cd beta-diversity/unifrac

qiime diversity beta-group-significance \
  --i-distance-matrix weighted-unifrac-rh.qza \
  --m-metadata-file ../../metadata.tsv \
  --m-metadata-column Plant_status \
  --o-visualization unweighted-unifrac-plant-status-significance.qzv \
  --p-pairwise

(qiime2-2019.10) (qiime2-2019.10) Saved Visualization to: unweighted-unifrac-plant-status-significance.qzv
(qiime2-2019.10) 

: 1

In [11]:
qiime tools view unweighted-unifrac-plant-status-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.(qiime2-2019.10) 